# 縦のオリジナルテーブル確認ノートブック
このノートでは **オリジナルの縦持ちテーブル**（例: `public.nf_loto_final` や `public.loto_final`）を確認します。

主なこと：
- `db_config.py` を自動検出して接続
- 該当テーブル候補をスキャン
- 先頭確認 / 列情報 / 行数 / 系列別の範囲
- 任意の `unique_id` の時系列プロット

必要なら `TABLE_ORIG` を手で上書きして使ってください。

In [1]:
# 0) セットアップ
import os, importlib, pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import matplotlib.pyplot as plt
print("pandas:", pd.__version__)


pandas: 2.3.3


In [2]:
# 1) db_config.py をロード（CWD → nf_featgen_bundle_v1/ → db/の順）
def _load_db_config():
    for p in [Path.cwd()/"db_config.py", Path.cwd()/"nf_featgen_bundle_v1"/"db_config.py", Path.cwd()/"db"/"db_config.py"]:
        if p.exists():
            spec = importlib.util.spec_from_file_location("db_config", str(p))
            mod = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(mod)
            print("Loaded:", p)
            return mod
    raise FileNotFoundError("db_config.py が見つかりません。")

db_cfg = _load_db_config()
DB_CONFIG = db_cfg.DB_CONFIG
TABLE_PREFIX = getattr(db_cfg, "TABLE_PREFIX", "")
DB_CONFIG


Loaded: /mnt/e/env/ts/test/db_config.py


{'user': 'postgres',
 'password': 'z',
 'host': 'localhost',
 'port': 5432,
 'dbname': 'postgres'}

In [3]:
# 2) SQLAlchemy エンジン作成
def make_url(cfg: dict) -> str:
    if "url" in cfg and cfg["url"]:
        return cfg["url"]
    user = cfg.get("user") or cfg.get("username")
    pwd  = cfg.get("password") or cfg.get("pwd") or cfg.get("pass")
    host = cfg.get("host", "localhost")
    port = int(cfg.get("port", 5432))
    db   = cfg.get("dbname") or cfg.get("database") or cfg.get("db")
    if not (user and pwd and db):
        raise ValueError(f"DB設定を確認してください。渡されたキー: {list(cfg.keys())}")
    return f"postgresql+psycopg2://{quote_plus(user)}:{quote_plus(pwd)}@{host}:{port}/{db}"

engine = create_engine(make_url(DB_CONFIG), pool_pre_ping=True, future=True)
with engine.begin() as conn:
    print(conn.execute(text("SELECT current_user, current_database();")).all())


[('postgres', 'postgres')]


In [4]:
# 3) テーブル候補を一覧（loto + final を含む名前）
import pandas as pd
from sqlalchemy import text

def q(sql, params=None, limit=None):
    if limit is not None and "limit" not in sql.lower():
        sql = sql.rstrip(";") + f" LIMIT {int(limit)}"
    return pd.read_sql(text(sql), engine, params=params)

cand = q('''
SELECT table_schema, table_name, (table_schema || '.' || table_name) AS full_name
FROM information_schema.tables
WHERE table_type='BASE TABLE'
  AND table_schema NOT IN ('pg_catalog','information_schema')
  AND (table_name ILIKE '%loto%final%' OR table_name IN ('loto_final','nf_loto_final'))
ORDER BY 1,2
''')
cand


,table_schema,table_name,full_name
0,public,nf_loto_final,public.nf_loto_final


In [5]:
# 4) TABLE_ORIG を決める（候補があれば最初を採用）。手で上書きOK。
if len(cand) > 0:
    TABLE_ORIG = cand.iloc[0]['full_name']
else:
    # 既定（必要に応じて編集）
    TABLE_ORIG = 'public.nf_loto_final'

TABLE_ORIG


'public.nf_loto_final'

In [6]:
# 5) 先頭確認
preview = q(f"SELECT * FROM {TABLE_ORIG} ORDER BY 1,2,3", limit=5)
preview


,loto,num,ds,unique_id,y,co,n1nu,n1pm,n2nu,n2pm,n3nu,n3pm,n4nu,n4pm,n5nu,n5pm,n6nu,n6pm,n7nu,n7pm
0,bingo5,1,2017-04-05,N1,1,0,2,8578800,51,265900,133,45900,593,16600,3797,2300,19682,600,183059,200
1,bingo5,1,2017-04-05,N2,10,0,2,8578800,51,265900,133,45900,593,16600,3797,2300,19682,600,183059,200
2,bingo5,1,2017-04-05,N3,13,0,2,8578800,51,265900,133,45900,593,16600,3797,2300,19682,600,183059,200
3,bingo5,1,2017-04-05,N4,19,0,2,8578800,51,265900,133,45900,593,16600,3797,2300,19682,600,183059,200
4,bingo5,1,2017-04-05,N5,23,0,2,8578800,51,265900,133,45900,593,16600,3797,2300,19682,600,183059,200


In [ ]:
# 6) 列情報
cols = q('''
SELECT c.ordinal_position AS pos, c.column_name, c.data_type, c.is_nullable
FROM information_schema.columns c
WHERE (c.table_schema || '.' || c.table_name) = :tbl
ORDER BY c.ordinal_position
''', {"tbl": TABLE_ORIG})
cols


In [ ]:
# 7) 行数
q(f"SELECT COUNT(*) AS n FROM {TABLE_ORIG}")


In [ ]:
# 8) 系列別の分布（unique_id ごとの範囲・件数）
dist = q(f"""SELECT unique_id, MIN(ds) AS ds_min, MAX(ds) AS ds_max, COUNT(*) AS n
FROM {TABLE_ORIG}
GROUP BY unique_id
ORDER BY unique_id
"""\)
dist.head(10)


In [ ]:
# 9) 任意の unique_id を1つ選び、y を時系列プロット
uid = q(f"SELECT DISTINCT unique_id FROM {TABLE_ORIG} ORDER BY 1 LIMIT 1").iloc[0,0]
df = q(f"""SELECT ds, y
FROM {TABLE_ORIG}
WHERE unique_id = :uid
ORDER BY ds
""" , {"uid": uid})
print("unique_id:", uid, "rows:", len(df))

plt.figure()
plt.plot(df["ds"], df["y"])    # seaborn禁止、色指定なし、単独プロット
plt.title(f"Original y over time - {uid}")
plt.xlabel("ds")
plt.ylabel("y")
plt.show()


In [ ]:
# 10) データ品質の軽い確認（重複キー、NULL 等）
checks = {
    'null_y': q(f"SELECT COUNT(*) AS null_y FROM {TABLE_ORIG} WHERE y IS NULL"),
    'dup_uid_ds': q(f"""        SELECT COUNT(*) AS dup_pairs FROM (
            SELECT unique_id, ds, COUNT(*) AS c
            FROM {TABLE_ORIG}
            GROUP BY unique_id, ds
            HAVING COUNT(*) > 1
        ) t
    """),
}
checks
